In [1]:
# download dependencies
!pip install yfinance==0.2.38
!pip install pandas==2.2.2
!pip install nbformat
!mamba install bs4==4.10.0



0 examples ran in 0.0003 seconds


In [28]:
# import modules
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [35]:
import plotly.io as pio
pio.renderers.default='iframe'

In [66]:
# define a graphing function
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [72]:
# Q1. extract Tesla stock data using yfinance
tesla = yf.Ticker("TSLA")
tsla_data = tesla.history(period="max")
tsla_data.reset_index(inplace=True)
tsla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [78]:
# Q2. extract Tesla revenue data from web using BeautifulSoup
tsla_url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
tsla_info = requests.get(tsla_url).text
tsla_soup = BeautifulSoup(tsla_info, "html5lib")
tsla_revenue = pd.DataFrame(columns=["Date", "Revenue"])
for row in tsla_soup.find("tbody").find_all("tr"):
    col = row.find_all("td")
    date = col[0].text
    revenue = col[1].text

    tsla_series = pd.Series({"Date": date, "Revenue": revenue})
    tsla_revenue = pd.concat([tsla_revenue, tsla_series.to_frame().T], ignore_index=True)

tsla_revenue["Revenue"] = tsla_revenue["Revenue"].str.replace("$", "").str.replace(",", "")

tsla_revenue.tail()

,Date,Revenue
8,2013,2013
9,2012,413
10,2011,204
11,2010,117
12,2009,112


In [74]:
# Q3. extract GameStop stock data using yfinance
gamestop = yf.Ticker("GME")
gme_data = gamestop.history(period="max")
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716073,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662209,1.603296,1.662209,6892800,0.0,0.0


In [79]:
# Q4. extract GameStop revenue data from web using read_html
from io import StringIO
gme_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html'
gme_info = requests.get(gme_url).text
gme_soup = BeautifulSoup(gme_info, "html5lib")
gme_revenue = pd.read_html(StringIO(str(gme_soup)))[0]
gme_revenue.columns = ["Date", "Revenue"]

gme_revenue["Date"] = gme_revenue["Date"].astype("string")
gme_revenue["Revenue"] = gme_revenue["Revenue"].str.replace("$", "").str.replace(",", "")

gme_revenue.tail()

,Date,Revenue
11,2009,8806
12,2008,7094
13,2007,5319
14,2006,3092
15,2005,1843


In [80]:
# Q5. graph Tesla stock and revenue dashboard
make_graph(tsla_data, tsla_revenue, "Tesla stock and revenue dashboard")

In [81]:
#Q6. graph GameStop stock and revenue dashboard
make_graph(gme_data, gme_revenue, "GameStop stock and revenue dashboard")